In [1]:
{-# LANGUAGE PackageImports #-}
{-# LANGUAGE OverloadedStrings #-}
import           "lens"                    Control.Lens
import qualified "bytestring"              Data.ByteString.Lazy.Char8 as BL8

In [2]:
import           "language-powerquery-ast" Language.PowerQuery.AST
import           "language-powerquery"     Language.PowerQuery
import           "pbix"                    Codec.Pbix.Types

In [3]:
bs <- BL8.readFile "/home/barak/Shared/PowerBI/csvTable/csvTable.pbix"

In [4]:
bs ^. pbix . dataMashup . formula "Section1.m"


Formula "section Section1;\r\n\r\nshared iris = let\r\n    Source = Csv.Document(File.Contents(\"\\\\VBOXSVR\\Shared\\PowerBI\\iris.csv\"),[Delimiter=\",\", Columns=5, Encoding=1252, QuoteStyle=QuoteStyle.None]),\r\n    #\"Promoted Headers\" = Table.PromoteHeaders(Source, [PromoteAllScalars=true]),\r\n    #\"Changed Type\" = Table.TransformColumnTypes(#\"Promoted Headers\",{{\"sepal_length\", type number}, {\"sepal_width\", type number}, {\"petal_length\", type number}, {\"petal_width\", type number}, {\"species\", type text}})\r\nin\r\n    #\"Changed Type\";"

In [6]:
fromRight (Right x) = x
let (Formula f) = bs ^. pbix . dataMashup . formula "Section1.m"
print f
parseDocument . fromRight . lexer . BL8.unpack $ f

"section Section1;\r\n\r\nshared iris = let\r\n    Source = Csv.Document(File.Contents(\"\\\\VBOXSVR\\Shared\\PowerBI\\iris.csv\"),[Delimiter=\",\", Columns=5, Encoding=1252, QuoteStyle=QuoteStyle.None]),\r\n    #\"Promoted Headers\" = Table.PromoteHeaders(Source, [PromoteAllScalars=true]),\r\n    #\"Changed Type\" = Table.TransformColumnTypes(#\"Promoted Headers\",{{\"sepal_length\", type number}, {\"sepal_width\", type number}, {\"petal_length\", type number}, {\"petal_width\", type number}, {\"species\", type text}})\r\nin\r\n    #\"Changed Type\";"

Parse error [IdentifierT (RegularI "number"),OperatorT RightCurlyO,OperatorT CommaO,OperatorT LeftCurlyO,LiteralT (StringL "\"sepal_width\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "number"),OperatorT RightCurlyO,OperatorT CommaO,OperatorT LeftCurlyO,LiteralT (StringL "\"petal_length\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "number"),OperatorT RightCurlyO,OperatorT CommaO,OperatorT LeftCurlyO,LiteralT (StringL "\"petal_width\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "number"),OperatorT RightCurlyO,OperatorT CommaO,OperatorT LeftCurlyO,LiteralT (StringL "\"species\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "text"),OperatorT RightCurlyO,OperatorT RightCurlyO,OperatorT RightParenO,KeywordT InK,IdentifierT (QuotedI "#\"Changed Type\""),OperatorT SemiColonO]
CallStack (from HasCallStack):
  error, called at dist/build/Language/PowerQuery/Parser.hs:2943:17 in language-powerquery-0.1.0.0-BpndezBezmwFpYMKNOnFER:Language.PowerQuery.Parser

In [7]:
fromRight (Right x) = x
parseLetExpression . fromRight . lexer $ "let\r\n    Source = Csv.Document(File.Contents(\"\\\\VBOXSVR\\Shared\\PowerBI\\iris.csv\"),[Delimiter=\",\", Columns=5, Encoding=1252, QuoteStyle=QuoteStyle.None]),\r\n    #\"Promoted Headers\" = Table.PromoteHeaders(Source, [PromoteAllScalars=true]),\r\n    #\"Changed Type\" = Table.TransformColumnTypes(#\"Promoted Headers\",{{\"sepal_length\", type number}, {\"sepal_width\", type number}, {\"petal_length\", type number}, {\"petal_width\", type number}, {\"species\", type text}})\r\nin\r\n    #\"Changed Type\";"

Parse error [LiteralT (StringL "\"\\\\VBOXSVR\\Shared\\PowerBI\\iris.csv\""),OperatorT RightParenO,OperatorT CommaO,OperatorT LeftBracketO,IdentifierT (RegularI "Delimiter"),OperatorT EqualO,LiteralT (StringL "\",\""),OperatorT CommaO,IdentifierT (RegularI "Columns"),OperatorT EqualO,LiteralT (NumberL (IntegerL 5)),OperatorT CommaO,IdentifierT (RegularI "Encoding"),OperatorT EqualO,LiteralT (NumberL (IntegerL 1252)),OperatorT CommaO,IdentifierT (RegularI "QuoteStyle"),OperatorT EqualO,IdentifierT (RegularI "QuoteStyle.None"),OperatorT RightBracketO,OperatorT RightParenO,OperatorT CommaO,IdentifierT (QuotedI "#\"Promoted Headers\""),OperatorT EqualO,IdentifierT (RegularI "Table.PromoteHeaders"),OperatorT LeftParenO,IdentifierT (RegularI "Source"),OperatorT CommaO,OperatorT LeftBracketO,IdentifierT (RegularI "PromoteAllScalars"),OperatorT EqualO,KeywordT TrueK,OperatorT RightBracketO,OperatorT RightParenO,OperatorT CommaO,IdentifierT (QuotedI "#\"Changed Type\""),OperatorT EqualO,IdentifierT (RegularI "Table.TransformColumnTypes"),OperatorT LeftParenO,IdentifierT (QuotedI "#\"Promoted Headers\""),OperatorT CommaO,OperatorT LeftCurlyO,OperatorT LeftCurlyO,LiteralT (StringL "\"sepal_length\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "number"),OperatorT RightCurlyO,OperatorT CommaO,OperatorT LeftCurlyO,LiteralT (StringL "\"sepal_width\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "number"),OperatorT RightCurlyO,OperatorT CommaO,OperatorT LeftCurlyO,LiteralT (StringL "\"petal_length\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "number"),OperatorT RightCurlyO,OperatorT CommaO,OperatorT LeftCurlyO,LiteralT (StringL "\"petal_width\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "number"),OperatorT RightCurlyO,OperatorT CommaO,OperatorT LeftCurlyO,LiteralT (StringL "\"species\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "text"),OperatorT RightCurlyO,OperatorT RightCurlyO,OperatorT RightParenO,KeywordT InK,IdentifierT (QuotedI "#\"Changed Type\""),OperatorT SemiColonO]
CallStack (from HasCallStack):
  error, called at dist/build/Language/PowerQuery/Parser.hs:2913:17 in language-powerquery-0.1.0.0-BpndezBezmwFpYMKNOnFER:Language.PowerQuery.Parser

In [20]:
lexer "x(#\"Promoted Headers\",{{\"sepal_length\", type number}, {\"sepal_width\", type number}, {\"petal_length\", type number}, {\"petal_width\", type number}, {\"species\", type text}})"

Right [IdentifierT (RegularI "x"),OperatorT LeftParenO,IdentifierT (QuotedI "#\"Promoted Headers\""),OperatorT CommaO,OperatorT LeftCurlyO,OperatorT LeftCurlyO,LiteralT (StringL "\"sepal_length\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "number"),OperatorT RightCurlyO,OperatorT CommaO,OperatorT LeftCurlyO,LiteralT (StringL "\"sepal_width\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "number"),OperatorT RightCurlyO,OperatorT CommaO,OperatorT LeftCurlyO,LiteralT (StringL "\"petal_length\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "number"),OperatorT RightCurlyO,OperatorT CommaO,OperatorT LeftCurlyO,LiteralT (StringL "\"petal_width\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "number"),OperatorT RightCurlyO,OperatorT CommaO,OperatorT LeftCurlyO,LiteralT (StringL "\"species\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "text"),OperatorT RightCurlyO,OperatorT RightCurlyO,OperatorT RightParenO]

In [21]:
lexer "x.x(#\"Promoted Headers\",{{\"sepal_length\", type number}, {\"sepal_width\", type number}, {\"petal_length\", type number}, {\"petal_width\", type number}, {\"species\", type text}})"

Left "lexical error at line 1, column 3"

In [28]:
lexer "#\"Changed Type\" = xxx(#\"Promoted Headers\",{{\"sepal_length\", type number}, {\"sepal_width\", type number}, {\"petal_length\", type number}, {\"petal_width\", type number}, {\"species\", type text}})"

Right [IdentifierT (QuotedI "#\"Changed Type\""),OperatorT EqualO,IdentifierT (RegularI "xxx"),OperatorT LeftParenO,IdentifierT (QuotedI "#\"Promoted Headers\""),OperatorT CommaO,OperatorT LeftCurlyO,OperatorT LeftCurlyO,LiteralT (StringL "\"sepal_length\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "number"),OperatorT RightCurlyO,OperatorT CommaO,OperatorT LeftCurlyO,LiteralT (StringL "\"sepal_width\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "number"),OperatorT RightCurlyO,OperatorT CommaO,OperatorT LeftCurlyO,LiteralT (StringL "\"petal_length\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "number"),OperatorT RightCurlyO,OperatorT CommaO,OperatorT LeftCurlyO,LiteralT (StringL "\"petal_width\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "number"),OperatorT RightCurlyO,OperatorT CommaO,OperatorT LeftCurlyO,LiteralT (StringL "\"species\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "text"),OperatorT RightCurlyO,OperatorT RightC

In [29]:
lexer "section Section1;\r\n\r\nshared iris = let\r\n    Source = CsvDocument(FileContents(\"\\\\VBOXSVR\\Shared\\PowerBI\\iris.csv\"),[Delimiter=\",\", Columns=5, Encoding=1252, QuoteStyle=QuoteStyleNone]),\r\n    #\"Promoted Headers\" = TablePromoteHeaders(Source, [PromoteAllScalars=true]),\r\n    #\"Changed Type\" = TableTransformColumnTypes(#\"Promoted Headers\",{{\"sepal_length\", type number}, {\"sepal_width\", type number}, {\"petal_length\", type number}, {\"petal_width\", type number}, {\"species\", type text}})\r\nin\r\n    #\"Changed Type\";"

Right [KeywordT SectionK,IdentifierT (RegularI "Section1"),OperatorT SemiColonO,KeywordT SharedK,IdentifierT (RegularI "iris"),OperatorT EqualO,KeywordT LetK,IdentifierT (RegularI "Source"),OperatorT EqualO,IdentifierT (RegularI "CsvDocument"),OperatorT LeftParenO,IdentifierT (RegularI "FileContents"),OperatorT LeftParenO,LiteralT (StringL "\"\\\\VBOXSVR\\Shared\\PowerBI\\iris.csv\""),OperatorT RightParenO,OperatorT CommaO,OperatorT LeftBracketO,IdentifierT (RegularI "Delimiter"),OperatorT EqualO,LiteralT (StringL "\",\""),OperatorT CommaO,IdentifierT (RegularI "Columns"),OperatorT EqualO,LiteralT (NumberL (IntegerL 5)),OperatorT CommaO,IdentifierT (RegularI "Encoding"),OperatorT EqualO,LiteralT (NumberL (IntegerL 1252)),OperatorT CommaO,IdentifierT (RegularI "QuoteStyle"),OperatorT EqualO,IdentifierT (RegularI "QuoteStyleNone"),OperatorT RightBracketO,OperatorT RightParenO,OperatorT CommaO,IdentifierT (QuotedI "#\"Promoted Headers\""),OperatorT EqualO,IdentifierT (RegularI "TablePromo

In [11]:
fromRight (Right x) = x
parseDocument . fromRight . lexer $ "section Section1;\r\n\r\nshared iris = let\r\n    Source = CsvDocument(FileContents(\"\\\\VBOXSVR\\Shared\\PowerBI\\iris.csv\"),[Delimiter=\",\", Columns=5, Encoding=1252, QuoteStyle=QuoteStyleNone]),\r\n    #\"Promoted Headers\" = TablePromoteHeaders(Source, [PromoteAllScalars=5]),\r\n    #\"Changed Type\" = TableTransformColumnTypes(#\"Promoted Headers\",{{\"sepal_length\", type number}, {\"sepal_width\", type number}, {\"petal_length\", type number}, {\"petal_width\", type number}, {\"species\", type text}})\r\nin\r\n    #\"Changed Type\";"

Parse error [IdentifierT (RegularI "number"),OperatorT RightCurlyO,OperatorT CommaO,OperatorT LeftCurlyO,LiteralT (StringL "\"sepal_width\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "number"),OperatorT RightCurlyO,OperatorT CommaO,OperatorT LeftCurlyO,LiteralT (StringL "\"petal_length\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "number"),OperatorT RightCurlyO,OperatorT CommaO,OperatorT LeftCurlyO,LiteralT (StringL "\"petal_width\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "number"),OperatorT RightCurlyO,OperatorT CommaO,OperatorT LeftCurlyO,LiteralT (StringL "\"species\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "text"),OperatorT RightCurlyO,OperatorT RightCurlyO,OperatorT RightParenO,KeywordT InK,IdentifierT (QuotedI "#\"Changed Type\""),OperatorT SemiColonO]
CallStack (from HasCallStack):
  error, called at dist/build/Language/PowerQuery/Parser.hs:2929:17 in language-powerquery-0.1.0.0-BpndezBezmwFpYMKNOnFER:Language.PowerQuery.Parser

In [34]:
fromRight (Right x) = x
parseExpression . fromRight . lexer $ "{\"sepal length\", type number}"
-- parseExpression . fromRight . lexer $ "x.x(#\"Promoted Headers\",{{\"sepal_length\", type number}, {\"sepal_width\", type number}, {\"petal_length\", type number}, {\"petal_width\", type number}, {\"species\", type text}})"

Parse error [LiteralT (StringL "\"sepal length\""),OperatorT CommaO,KeywordT TypeK,IdentifierT (RegularI "number"),OperatorT RightCurlyO]
CallStack (from HasCallStack):
  error, called at dist/build/Language/PowerQuery/Parser.hs:2913:17 in language-powerquery-0.1.0.0-BpndezBezmwFpYMKNOnFER:Language.PowerQuery.Parser

In [9]:
fromRight (Right x) = x
parseDocument . fromRight . lexer $ "section Bobo1; shared iris = let Source = Csv.Document(File.Contents(\"hello world\")) in y;"

SectionDocument (Section {_section_attributes = Nothing, _section_name = Just (RegularI "Bobo1"), _section_members = Just [SectionMember {_sectionMember_attributes = Nothing, _sectionMember_shared = True, _sectionMember_name = RegularI "iris", _sectionMember_expression = LetE (LetExpression {_letExpression_variableList = [Variable {_variable_name = RegularI "Source", _variable_expression = LogicalE (And_OE (Is_LAE (As_IE (EqualityAE (RelationalEE (AdditiveRE (MultiplicativeAE (MetadataME (MetadataExpression {_metadataExpression_first = UnaryType (Primary_TE (InvokePE (InvokeExpression {_invokeExpression_primary = FieldAccessPE (ImplicitTargetProjection {_implicitTargetProjection_identifier = RegularI "Csv.Document", _implicitTargetProjection_annotation = Just Annotation}), _invokeExpression_argumentList = Just [LogicalE (And_OE (Is_LAE (As_IE (EqualityAE (RelationalEE (AdditiveRE (MultiplicativeAE (MetadataME (MetadataExpression {_metadataExpression_first = UnaryType (Primary_TE (Invok